# Лабораторная работа 3. Обучение без учителя

Результат лабораторной работы − отчет. Мы предпочитаем принимать отчеты в формате ноутбуков IPython (ipynb-файл). Постарайтесь сделать ваш отчет интересным рассказом, последовательно отвечающим на вопросы из заданий. Помимо ответов на вопросы, в отчете также должен быть код, однако чем меньше кода, тем лучше всем: нам − меньше проверять, вам — проще найти ошибку или дополнить эксперимент. При проверке оценивается четкость ответов на вопросы, аккуратность отчета и кода.

## Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 15 баллов. Сдавать задание после указанного срока сдачи нельзя. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов и понижают карму. Если вы нашли решение какого-то из заданий в открытом источнике, необходимо прислать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, нам необходима ссылка на источник).

## EM-алгоритм

### Краудсорсинг

Разметка данных — одна из самых трудозатратных задач в машинном обучении. Краудсорсинг позволяет распределить эту задачу на тысячи исполнителей, каждый из которых подготавливает небольшую часть датасета (подробнее <a href="https://academy.yandex.ru/posts/chto-takoe-kraudsorsing-i-pochemu-emu-nuzhno-uchitsya">тут</a>).

Пользователи могут допускать ошибки при разметке, кроме того, среди пользователей могут быть боты. Если мы попросим разметить каждый объект только одного пользователя, то с большой вероятностью получим не достаточно качественную разметку. Обычно каждый объект размечают несколько пользователей.

Результаты разметки нужно обработать. Самый простой метод *голос большинства*. Он заключается в том, что для каждого объекта нужно взять тот класс, который чаще всего ставили пользователи данному объекту. Это достаточно хороший метод, но он не учитывает различные особенности пользователей. Далее рассмотрим метод, который позволяет оценивать вероятность того, что разметчик ошибся.

### Метод Дэвида-Скина (Dawid, Skene, 1979)

Мы имеем в качестве данных $n_{ik}^u$ &mdash; количество раз, при которых разметчик $u \in U$ поставил класс $k \in K$ объекту $i \in I$ (возможно, разметчик видел этот объект несколько раз). Обозначим $Y_{ik} = I\{\text{объект $i$ класса $k$}\}$, это наши латентные величины. 

В качестве параметров имеем
* $\pi_{k\ell}^u$ &mdash; вероятность того, что разметчик $u$ поставил класс $\ell$ вместо правильного класса $k$. 
* $\rho_k$ &mdash; вероятность класса $k$.

Поймём, какой будет функция неполного правдоподобия в этой задаче. Прежде всего,

$$p_{\pi,p}(N, Y) = \prod_{i\in I}p(N_i, Y_i),$$

Если $k$ - номер класса $i$-го объекта, то

$$p(N_i, Y_i)=\underbrace{p(\text{объект $i$ класса $k$})}_{=\rho_k}p(N_i\mid\text{объект $i$ класса $k$})$$

(значения $Y_{it}$ однозначно определяются номером истинного класса, поэтому справа $Y_i$ пропадает). Далее, мы считаем, что разметчики действуют независимо, поэтому

$$p(N_i\mid\text{объект $i$ класса $k$}) = \prod_{u\in U}p(N_i^u\mid\text{объект $i$ класса $k$}).$$

Разберёмся с величиной $p(N_i^u\mid\text{объект $i$ класса $k$})$. Она отвечает за то, какие классы $u$-й разметчик ставил $i$-му объекту. Мы считаем, что встречи разметчика с объектом упорядочены по времени, тогда

$$p(\text{$u$-й разметчик отнёс $i$-й объект к классам $k'_1,\ldots,k'_r$}\mid\text{объект $i$ класса $k$}) =$$

$$=\prod_{s}p(\text{в $s$-ю встречу с $i$-м объектом $u$-й разметчик отнёс его к классу $k'_s$}\mid\text{объект $i$ класса $k$})$$

Эту вероятность можно переписать в виде

$$\prod_{\ell \in K} \left( \pi_{k\ell}^u \right)^{n_{i\ell}^u},$$

а итоговое неполное правдоподобие предстаёт в виде

$$p_{\pi,p}(N, Y) = \prod_{i\in I}\prod_{k \in K} \left( \rho_k \prod_{u\in U} \prod_{\ell \in K} \left( \pi_{k\ell}^u \right)^{n_{i\ell}^u} \right)^{Y_{ik}}$$

Его нам нужно максимизировать по $\pi$ и $\rho$

**Пояснение к формуле:** 

Вне больших скобок фиксируются объект и его класс, сама скобка возводится в степень 1, если рассматривается правильный класс объекта, и в степень 0 иначе. Внутри сначала записана вероятность того, что объект имеет данный класс, а затем &mdash; перебор по всем пользователям и всем классам, которые мог поставить данный пользователь. Наконец, записывается вероятность того, что пользователь нашему объекту поставил некоторый класс, которая возводится в степень того, сколько раз он поставил этот класс. Например, если пользователь видел изображение котика 5 раз, при этом 3 раза он сказал, что котик, а два раза &mdash; песик, то вероятность $\pi_{cat,cat}^u$ для данного котика учтется 3 раза, а вероятность $\pi_{cat,dog}^u$ &mdash; 2 раза.

In [ ]:
!pip install crowd-kit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 587.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.0 MB/s eta 0:00:00


In [ ]:
from crowdkit.datasets import load_dataset

In [ ]:
df, df_gt = load_dataset('relevance-5')

Unpacking relevance-5.zip


In [ ]:
from crowdkit.aggregation import DawidSkene

In [ ]:
agg_ds = DawidSkene(n_iter=10).fit_predict(df)

In [ ]:
agg_ds.sort_index().to_csv("./out5DS.csv")

**Задание 1 (2 балл)**

Распишите итерационную процедуру EM-алгоритма и значение ELBO в методе Дэвида-Скина.

In [ ]:
class DawidSkene:
    def __init__(self, crowd_labels: pd.DataFrame):
        self.crowd_labels = crowd_labels
    
    def initialize_params(self):
        # your code here
    
    def predict(self):
        # your code here
        
    def run(self, n_iter):
        # your code here

**Задание 2 (3 балла)** 

Реализуйте следующие методы агрегации результатов разметки в краудсорсинге:

* голосование по большинству;
* метод Дэвида-Скина.

Оба метода должны возвращать вероятность принадлежности объекта каждому из классов (итоговая метка получается выбором класса с наибольшей оценкой вероятности). 

Заметим, что метод голосования по большинству можно реализовать с помощью одной агрегирующей функцией `pandas.crosstab`, а метод Дэвида-Скина основывается на EM-алгоритме (при реализации стоит учесть, что EM-алгоритм сходится в локальные оптимумы, то есть его стоит запускать из разных начальных приближений).

Примените эти два метода к датасетам *Toloka Aggregation Relevance 2* и *Toloka Aggregation Relevance 5*, которые можно скачать <a href="https://toloka.ai/ru/datasets">тут</a>, и сравните их между собой. 

Обратите внимание, что в последнем датасете 5 различных меток, причем некоторые объекты в датасете имеют не один, а несколько правильных ответов. Любой из таких ответов считается правильным.

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as sps
from sklearn.mixture import GaussianMixture
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set(font_scale=1.3)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
t2_cr = pd.read_table("./t2/crowd_labels.tsv", header=None)
t2_gold = pd.read_table("./t2/golden_labels.tsv", header=None)

In [ ]:
t5_cr = pd.read_table("./t5/crowd_labels.tsv", header=None)
t5_gold = pd.read_table("./t5/golden_labels.tsv", header=None)
t5_bans = pd.read_table("./t5/bans.tsv", header=None)

In [ ]:
t5_cr.head()

,0,1,2
0,w360,t186849,4
1,w43,t174611,5
2,w1040,t198665,3
3,w618,t63171,5
4,w98,t141574,5


In [ ]:
cross = pd.crosstab(t5_cr[1], t5_cr[2])

In [ ]:
cross.head()

2,1,2,3,4,5
1,,,,,
t0,0,0,3,0,0
t1,0,0,0,1,2
t10,0,0,0,0,3
t100,0,0,0,1,2
t1000,0,0,0,0,3


In [ ]:
cross[5]

1
t0        0
t1        2
t10       3
t100      2
t1000     3
         ..
t99995    0
t99996    0
t99997    3
t99998    3
t99999    3
Name: 5, Length: 363814, dtype: int64

In [ ]:
res_pred = []
res_obj = []
i = 0
for elem in cross.index:
    if i % 1000 == 0:
        print(i)
    res_pred.append(
        helper(cross[1][i], cross[2][i], cross[3][i], cross[4][i], cross[5][i])
    )
    res_obj.append(elem)
    i += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [ ]:
import scipy

In [ ]:
scipy.stats.mode(t2_gold[t2_gold[0]=="t0"][1])

ModeResult(mode=array([], dtype=float64), count=array([], dtype=float64))

In [ ]:
df = pd.DataFrame({'object': res_obj,
                   'prediction': res_pred}).sort_index()
df.to_csv("./out2.csv", index=False)

In [ ]:
def helper(x1, x2, x3, x4, x5):
    maxim = max(x1, x2, x3, x4, x5)
    if maxim == x1:
        return 1
    if maxim == x2:
        return 2
    if maxim == x3:
        return 3
    if maxim == x4:
        return 4
    if maxim == x5:
        return 5

In [ ]:
df.sort_values(by=['object'], key=lambda col: helper(col)).to_csv("./out2New.csv", index=False)

In [ ]:
t2_gold[t2_gold[0]=="t30006"]

,0,1
0,t30006,0


**В Я.Контест нужно загрузить только .csv файлы в следующем формате.**

,prediction
object,
t0,1
t1,1
t10,1
t100,0
t1000,0


In [ ]:
def majority_vote_agg(df: pd.DataFrame) -> pd.DataFrame:
    # your code here
    return result

**Задание 3 (1 балл)** 

Попробуйте в методе Дэвида-Скина в качестве начального приближения вероятностей классов для каждого объекта подавать те вероятности, которые посчитаны методом голосования по большинству, и провести сначала M-шаг.

In [ ]:
class DawidSkeneWithMVInit(DawidSkene):
    def initialize_params(self):
        # your code here

In [ ]:
import numpy as np

In [ ]:
a = np.array([[5,5,5],[5,5,5],[5,5,5],[5,5,5]])
c = np.array([[4,4,4,4,4,4],[4,4,4,4,4,4],[4,4,4,4,4,4]])
b = np.array([5,5,5,5])
a * b / b[:, np.newaxis, np.newaxis]

array([[[1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.]]])

Далее мы будем работать с таблицей `vacancies.csv` и нам нужно подготовить данные для работы, а именно сделайте следующее:
    
- Разбейте данные на обучающую выборку (строки, не содержащие метки кластеров) и тестовую (строки, содержащие метки кластеров)
- Предобработайте текст, содержащийся в колонках *name* и *description* (уберите артефакты, нормализуйте и т.д.)

### Тематическое моделирование

Тематическое моделирование заключается в поиске тем $T$, которые хорошо бы описывали документы $D$ со словарём $W$. Большинство тематических моделей оперирует данными в формате "мешка слов", т.е. учитывают только частоты слов в документах, а не их порядок. Одной из простейших тематических моделей является [PLSA](https://en.wikipedia.org/wiki/Probabilistic_latent_semantic_analysis), которая приводит к задаче стохастического матричного разложения: 

$$F \approx \Phi \times \Theta$$
где
- $F_{W \times D}$— матрица распределений слов в документах (нормированные частоты)
- $\Phi_{W \times T}$ — матрица распределений слов в темах (модель)
- $\Theta_{T \times D}$ — матрица распределений тем в документах (результат применения модели к обучающим данным)

Можно сказать, что алгоритмы тематического моделирования производят мягкую бикластеризацию данных:
 - *мягкую*, так как объекты относятся не строго к одному кластеру, а к нескольким с разными вероятностями
 - *бикластеризацию*, так как модель одновременно кластеризует слова по темам и темы по документам.
 
 С вероятностной точки зрения, задача обучения модели PLSA ставится как максимизация неполного правдоподобия по параметам $\Phi$ и $\Theta$. ЕМ-алгоритм для модели PLSA заключается в повторении двух шагов:

- **Е-шаг** — оценка распределений тем для каждого слова в каждом документе по параметрам $\Phi$ и $\Theta$ (шаг 6);
- **М-шаг** — обновление параметров $\Phi$ и $\Theta$ на основе полученных оценок (шаги 7 и 9).

Существуют различные модификации итерационного процесса, позволяющие снизить расходы по памяти. В данном случае, мы избежим хранения трехмерной матрицы $p_{tdw}$, сразу пересчитывая $\Theta$ для текущего документа и аккумулируя счетчики $n_{wt}$ для последующего пересчета $\Phi$.

Псевдокод алгоритма записывается следующим образом:

1. Инициализировать $\phi_{wt}^0$ для всех $w \in W$, $t \in T$ и $\theta_{td}^0$ для всех $t \in T$, $d \in D$
2. Внешний цикл по итерациям $i = 1 ... max\_iter$:
3. $\quad$ $n_{wt}^i := 0$, $n_t^i := 0$ для всех $w \in W$ и $t \in T$ 
4. $\quad$ Внутренний цикл по документам $d \in D$  
5. $\qquad$ $Z_w := \sum_{t \in T} \phi_{wt}^{i-1}\theta_{td}^{i-1}$ для всех $w \in d$ $\cfrac{}{}$
6. $\qquad$ $p_{tdw} := \cfrac{ \phi_{wt}^{i-1}\theta_{td}^{i-1} }{ Z_w }$ (**E-шаг**)
7. $\qquad$ $\theta_{td}^{i} := \cfrac{ \sum_{w \in d} n_{dw} p_{tdw} }{ n_d }$ для всех $t \in T$ (**M-шаг**)
8. $\qquad$ Увеличить $n_{wt}^i$ и $n_t^i$ на $n_{dw} p_{tdw}$ для всех $w \in W$ и $t \in T$
9. $\quad \phi_{wt}^i := \cfrac{n_{wt}^i}{n_t^i}$ для всех $w \in W$ и $t \in T$ (**M-шаг**)

Обозначения:
 - $p_{tdw}$ — вероятность темы $t$ для слова $w$ в документе $d$
 - $\phi_{wt}$ — элемент матрицы $\Phi$, соответствующий вероятности слова $w$ в теме $t$
 - $\theta_{td}$ — элемент матрицы $\Theta$, соответствующий вероятности темы $t$ в документе $d$
 - $n_{wt}$ — элемент матрицы счётчиков отнесения слова $w$ к теме $t$ (путем нормирования этой матрицы получается матрица $\Phi$)
 - $Z_w$ — элемент вектора вспомогательных переменных, соответствующий слову $w$
 - $n_t$ — вектор нормировочных констант для матрицы $n_{wt}$
 - $n_d$ — вектор нормировочных констант для матрицы $n_{dw}$
 - $n$ — суммарное число слов в коллекции

Для оценивания качества построенной модели и контроля сходимости процесса обучения обычно используют [перплексию](http://www.machinelearning.ru/wiki/images/8/88/Voron-iip9-talk.pdf):

$$\mathcal{P} = \exp\bigg(- \frac{\mathcal{L}}{n} \bigg) = \exp\bigg(- \cfrac{1}{n}\sum_{d \in D}\sum_{w \in d} n_{dw} \ln \big(\sum_{t \in T}\phi_{wt}\theta_{td} \big)\bigg)$$

Это традиционная мера качества в тематическом моделировании, которая основана на правдоподобии модели $\mathcal{L}$. Число итераций $max\_iter$ в алгоритме обучения следует выбирать достаточным для того, чтобы перплексия перестала существенно убывать. Однако известно, что перплексия плохо отражает интерпретируемость найденных тем, поэтому помимо нее обычно используются дополнительные меры или экспертные оценки.

**Рекомендации к реализации:**

- При делении на нулевые значения нужно просто заменить частное на ноль.
- ЕМ-алгоритм стоит реализовывать с использованием векторных операций. Для проверки корректности реализации сперва можно написать скалярную версию, после чего векторизовать её, удостоверившись, что обе реализации дают одинаковый результат. Невекторизованный алгоритм может работать в сотни раз медленнее векторизованного, и его использование может привести к невозможности выполнения задания.
- Итерационный процесс следует начинать, инициализировав матрицы $\Phi$ и $\Theta$. Инициализация может быть случайной, важно не забыть отнормировать столбцы матриц.
- Неэффективная реализация перплексии может в разы замедлить работу алгоритма.

**Задание 4 (3 балла)** 

Реализуйте описанный выше ЕМ-алгоритм для модели *PLSA* и добавьте в вашу реализацию подсчёт перплексии. 

Примените ваш алгоритм к подготовленным ранее данным (объедините текст из колонок *name* и *description*), рассмотрев число тем T = 5, а также:

* Постройте график значения перплексии в зависимости от итерации (убедитесь в корректности реализации: график перплексии должен быть невозрастающим). 
* Выведите для каждой темы топ-20 наиболее вероятных слов.

Посмотрите внимательно на полученные темы. Как вам кажется, получились ли они интерпретируемыми?

In [ ]:
class PLSA:
    def __init__(self, counts: np.matrix, T: int):
        self.counts = counts
        self.T = T
        self.initialize_params()
    
    def initialize_params(self):
        self.Phi = np.random.dirichlet(np.ones(self.T), size=self.counts.shape[0])
        self.Theta = np.random.dirichlet(np.ones(self.counts.shape[1]), size=self.T)

    def step(self):
        N = np.zeros((self.counts.shape[0], self.T))
        n = np.zeros(self.T)
        divider = self.Phi @ self.Theta
        p = np.zeros((self.T, self.counts.shape[0]))
        for d in range(self.counts.shape[1]):
            theta = self.Theta[:,d]
            p = (self.Phi * theta[np.newaxis, :]).T / divider[:,d]
            self.Theta[:,d] = p @ self.counts[:, d] / np.sum(self.counts[:, d])
            N = N + p.T * self.counts[:, d][:,np.newaxis]
            n = n + p @ self.counts[:, d]
        self.Phi = N / n[np.newaxis,:]

    def perplexity(self) -> float:
        X = self.counts
        score = (X * np.log(self.Phi @ self.Theta)).sum()
        score /= X.sum()
        
        return np.exp(-score)

In [ ]:
p = np.array([[1,2,3],[4,5,6],[7,8,9],[10,20,30]])
p[:,1] = p[:, 2]
p

array([[ 1,  3,  3],
       [ 4,  6,  6],
       [ 7,  9,  9],
       [10, 30, 30]])

**Задание 5 (1 балл)** 

Рассмотрите большее число тем (10, 20) и несколько различных начальных приближений. Проанализируйте результаты и ответьте на следующие вопросы: 

- Mожно ли сказать, что интерпретируемость каждой темы изменяется с ростом их числа?
- Устойчив ли алгоритм к начальному приближению на примере идентичности топовых слов в соответствующих темах?
- Отражает ли перплексия качество получаемых моделей? В чём заключается причина хорошего/плохого соответствия?

## Кластеризация

**Задание 6 (2 балла)** 

В данном задание следуют сравнить между собой алгоритмы [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html), [DBSCAN](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) и [Birch](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.Birch.html) используя подготовленные ранее данные.

Поэксперементируйте с различными способами векторизации текста, например можно:

- использовать только *name* / *description*, объединить их в один текст или сконкатенировать векторные представления,
- использовать представление в виде мешка слов или какую-либо его модификацию,
- сократить размерность векторного представления, используя PCA или векторные представления слов,

или сделать что-то более интересное, что вы придумаете!

Выберете лучшую комбинацию векторизации и алгоритма кластеризации и визуализируйте полученные кластеры (например, воспользовавшись облаком тегов, или предложите свой способ). Обоснуйте почему вы считаете, что выбранный вами подход для решения задачи кластеризации вакансий является лучшим.

**Задание "со звездочкой" (1 балл)** 

Попробуйте обогатить векторное представление полученное из текстов другими признаками из таблицы `vacancies.csv` и добиться лучшей интерпретируемости кластеров.

## Частичное обучение

Часто у нас есть размеченная выборка только для небольшой части выборки. Тогда мы можем применить подходы _частичного обучения (semi-supervised learning)_. Более подробно про реализацию таких методов в sklearn можно прочитать в разделе [semi-supervised](http://scikit-learn.org/stable/modules/label_propagation.html#semi-supervised).

**Задание 7 (2 балла)** 

В этом задание нужно сделать следующее: 

- Разделите объекты, у которых существуют метки, на обучающую и тестовую выборки (при этом не обязательно делить в соотношении 70% на 30%). Обогатите обучающую выборку объектами без меток.
- Воспользовавшись опытом выполнения задания 6, возьмите "лучшее" векторное представление вакансий и обучите [LabelSpreading](http://scikit-learn.org/stable/modules/generated/sklearn.semi_supervised.LabelSpreading.html) (подберите лучшие параметры, опираясь на [F1 score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)).
- Попробуйте запустить алгоритм несколько раз, отмечая известными различные объекты, а также меняя пропорции разбиения, посчитайте качество и визуализируйте результаты. Можно ли сказать что алгоритм сильно зависит от известных начальных объектов? Есть ли класс, для которого это больше всего заметно?